In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# PREPARATION PROCESS

Constraint:

1. Two or more activities can't share the same teacher or the same students sets in a single schedule
2. Maximum number of classroom and laboratorium used in a single schedule is 28 and 10, respectively

In [ ]:
# Input data from csv (comma-separated value) file, and maximum number of classroom and laboratorium available
df = pd.read_csv("Data/FET ALL_LAB_KELAS.csv")
max_class = int(input("Enter the maximum number of classroom: "))
max_lab = int(input("Enter the maximum number of laboratorium: "))
df

In [ ]:
# Initialize the graph with zeros
Graph = np.zeros((len(df), len(df)), dtype=int)
network = nx.Graph()
network.add_nodes_from(df.index) # Initiatiate nodes for the visualisation
print(f"This network has {network.number_of_nodes()} nodes")
Graph

In [ ]:
# Assign value to the adjacency matrix by the connectivity of the nodes
for i in range(len(df)):
    for j in range(i+1, len(df)):
        if df.loc[i, "Students Sets"] == df.loc[j, "Students Sets"] or df.loc[i, "Teachers"] == df.loc[j, "Teachers"]:
            Graph[i][j] = 1
            Graph[j][i] = 1
            network.add_edge(i, j) # Add edges for the visualisation based on the rule
Graph

In [ ]:
# Graph visualisation before colorized
color_map = ["purple"]*(network.number_of_nodes())
pos = nx.spring_layout(network, seed = 1)
fig, ax = plt.subplots(figsize=(50,50))
ax = nx.draw_networkx(network, pos = pos, with_labels=True, node_size=1000, width=1, node_color=color_map, font_size=12, font_color="white", ax=ax)
fig.savefig("Result/graph-before-colorized.png", bbox_inches='tight')

In [ ]:
# Calculate the degree of each node, here's node as the key and its degree as the value of dictionary
nodeDegree = {}
for i in range(len(Graph)):
    nodeDegree[i] = sum(Graph[i])
nodeDegree

In [ ]:
# Sort the nodes by its degree in descending order
pairedSortedNode = sorted(nodeDegree.items(), key=lambda x: x[1], reverse=True) # Sort by dictionary value (node's degree)
sortedNode = np.zeros((len(df)), dtype=int) # Initiate numpy array by zero before assigning sorted node
for i in range(len(df)): 
    sortedNode[i] = pairedSortedNode[i][0] # Assign dictionary key (node) to sorted node list
sortedNode

In [ ]:
colorClass = 1 # color for class
colorLab = 1 # color for laboratorium
color_list = ["lightcoral", "gray", "lightgray", "firebrick", "red", "chocolate", "darkorange", "moccasin", "gold", "yellow", "darkolivegreen", "chartreuse", "forestgreen", "lime", "mediumaquamarine", "turquoise", "teal", "cadetblue", "dogerblue", "blue", "slateblue", "blueviolet", "magenta", "lightsteelblue"] # List of possible color (it has to be equal to the number of nodes (for the worst case), but it's okay because we only need this for the visualisation)

# Main Process

In [ ]:
# Function to check whether a new node is not adjacent to every single element of a given list
def isNotAdjacent(x, _list, _graph):
    for i in _list:
        if _graph[x][i] == 1:
            return False
    return True

In [ ]:
# Welch-Powell Algorithm, this algorithm runs in O(N*N) time complexity
colorizedNode = {}
listOfColorizedNode = []
for i in sortedNode:
    if i not in listOfColorizedNode:
        listOfColorizedNode.append(i)
        listOfAdjacentNode = []
        if df.loc[i, "Room"] == "KELAS":
            counterClass, counterLab = 1, 0
            colorizedNode[i] = colorClass
            color_map[i] = color_list[colorClass]
        elif df.loc[i, "Room"] == "LAB":
            counterClass, counterLab = 0, 1
            colorizedNode[i] = colorLab
            color_map[i] = color_list[colorLab]
        for j in sortedNode:
            if Graph[i][j] == 0 and i != j and isNotAdjacent(j, listOfAdjacentNode, Graph) == True and j not in listOfColorizedNode and df.loc[j, "Room"] == "KELAS" and counterClass < max_class:
                colorizedNode[j] = colorClass
                color_map[j] = color_list[colorClass]
                listOfColorizedNode.append(j)
                listOfAdjacentNode.append(j)
                counterClass = counterClass + 1
            elif Graph[i][j] == 0 and i != j and isNotAdjacent(j, listOfAdjacentNode, Graph) == True and j not in listOfColorizedNode and df.loc[j, "Room"] == "LAB" and counterLab < max_lab:
                colorizedNode[j] = colorLab
                color_map[j] = color_list[colorLab]
                listOfColorizedNode.append(j)
                listOfAdjacentNode.append(j)
                counterLab = counterLab + 1
            elif counterClass == max_class and counterLab == max_lab:
                break
        listOfAdjacentNode *= 0
        colorLab = colorLab + 1
        colorClass = colorClass + 1
colorizedNode

In [ ]:
for t,w in sorted(colorizedNode.items()):
  print("Node",t,"  \t=","Color", w)

In [ ]:
# Activity is the colorized node, schedule is the color of a node
schedule_list = []
for activity, schedule in sorted(colorizedNode.items()): # Assign the sorted colorized node by node's name (Activity ID) to a new list
    schedule_list.append(schedule)
df["Schedules"] = schedule_list # Add new column of schedule of an activity
df.head(50)

In [ ]:
df = df.sort_values("Schedules") # Sort the data frame by the value of schedules (node's color)
df["Schedules"] = df["Schedules"].astype(str) # Type conversion of the schedules column from integer to string
df

In [ ]:
# Dummy column ("String" column), so that another column can be concatenated with "Schedule" string
schedule_string = ["Schedule"]*len(df)
df["String"] = schedule_string
df

In [ ]:
# Concatenate "String" column with "Schedules" column so that it can be more descriptic, and then drop the "String" column
df["Schedules"] = df["String"] + " " + df["Schedules"]
df.drop("String", axis="columns", inplace=True)
df

In [ ]:
# Export dataframe to csv (comma-separated value) file
df.to_csv("Result/schedule-result.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(50,50))
nx.draw_networkx(network, pos = pos, with_labels=True, node_size=1000, width=1, node_color=color_map, font_size=12, font_color="white", ax=ax)
fig.savefig("Result/graph-after-colorized.png", bbox_inches='tight')